# DALI CLAHE (Contrast Limited Adaptive Histogram Equalization) Example

This notebook demonstrates how to use **CLAHE (Contrast Limited Adaptive Histogram Equalization)** in a DALI pipeline for image preprocessing. CLAHE is a powerful technique that improves contrast in images without overamplifying noise, making it particularly useful for medical imaging, surveillance, and low-contrast photography.

## What is CLAHE?

CLAHE is an enhanced version of Adaptive Histogram Equalization (AHE) that:
- ✅ **Improves local contrast** by processing image tiles independently
- ✅ **Prevents noise amplification** through contrast limiting
- ✅ **Preserves image quality** while enhancing details
- ✅ **Works well on various image types** from medical scans to natural photos

## Key Features of DALI's CLAHE Implementation

- 🚀 **High Performance**: GPU-accelerated implementation
- 🔧 **Flexible Parameters**: Customizable tile sizes and clip limits
- 🎨 **Color-Aware**: Option to process luminance only for RGB images
- 📊 **Seamless Integration**: Works within DALI pipelines

**References:**
- [Wikipedia: Adaptive Histogram Equalization](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization)
- [OpenCV CLAHE Tutorial](https://docs.opencv.org/4.12.0/d5/daf/tutorial_py_histogram_equalization.html)

## 📦 Required Imports

Let's start by importing the necessary DALI modules and NumPy for data analysis.

In [ ]:
import nvidia.dali as dali
import nvidia.dali.fn as fn
import nvidia.dali.types as types
import numpy as np

## 🏗️ Building the CLAHE Pipeline

The main pipeline function creates a DALI processing pipeline that applies CLAHE enhancement to images. This pipeline can work with either real images from a directory or synthetic test data.

### Key CLAHE Parameters:

| Parameter | Description | Typical Values |
|-----------|-------------|----------------|
| `tiles_x`, `tiles_y` | Grid size for local processing | 4-16 (higher = more local adaptation) |
| `clip_limit` | Threshold to prevent noise amplification | 1.0-4.0 (higher = more contrast) |
| `luma_only` | For RGB: process only luminance channel | `True` (preserves color balance) |

In [ ]:
def create_clahe_pipeline(
    batch_size=4, num_threads=2, device_id=0, image_dir=None
):
    """
    Create a DALI pipeline with CLAHE operator.

    Args:
        batch_size (int): Number of images per batch
        num_threads (int): Number of worker threads
        device_id (int): GPU device ID
        image_dir (str): Directory containing images (if None, uses synthetic data)

    Returns:
        DALI pipeline with CLAHE preprocessing
    """

    @dali.pipeline_def(
        batch_size=batch_size, num_threads=num_threads, device_id=device_id
    )
    def clahe_preprocessing_pipeline():
        if image_dir:
            # Read images from directory
            images, labels = fn.readers.file(
                file_root=image_dir, random_shuffle=True
            )
            images = fn.decoders.image(images, device="mixed")  # Decode on GPU

            # Resize to consistent size
            images = fn.resize(images, size=[256, 256])
        else:
            # Create synthetic test images with varying contrast
            # This simulates real-world scenarios where CLAHE is beneficial

            # Generate base image with moderate values to avoid overflow
            images = fn.random.uniform(
                range=(60, 180), shape=(256, 256, 3), dtype=types.FLOAT
            )

            # Add some contrast variation to make CLAHE effect visible
            contrast_factor = fn.random.uniform(range=(0.5, 0.9))
            images = images * contrast_factor

            # Add small brightness variation (keeping within safe range)
            brightness_offset = fn.random.uniform(range=(-20, 20))
            images = images + brightness_offset

            # Convert to uint8 (DALI will automatically clamp to [0,255])
            images = fn.cast(images, dtype=types.UINT8)

        # 🎯 Apply CLAHE for adaptive histogram equalization
        # This is where the magic happens!
        clahe_images = fn.clahe(
            images,
            tiles_x=8,  # 8x8 grid of tiles for local processing
            tiles_y=8,
            clip_limit=2.0,  # Moderate clipping to prevent noise
            luma_only=True,  # RGB: process luminance only to preserve colors
        )

        return images, clahe_images

    return clahe_preprocessing_pipeline()

## 🔬 Parameter Comparison Function

Let's create a function to demonstrate how different CLAHE parameters affect the results. This will help you understand how to tune the parameters for your specific use case.

In [ ]:
def demonstrate_clahe_parameters():
    """
    Demonstrate different CLAHE parameter settings to show their effects.

    Returns:
        DALI pipeline that generates one base image and three CLAHE variants
    """

    @dali.pipeline_def(batch_size=1, num_threads=1, device_id=0)
    def parameter_demo_pipeline():
        # Create a test image with poor contrast (narrow intensity range)
        base_image = fn.random.uniform(
            range=(80, 120), shape=(256, 256, 1), dtype=types.UINT8
        )

        # 🔧 Different CLAHE configurations to compare:

        # 1. Default settings - balanced approach
        clahe_default = fn.clahe(
            base_image,
            tiles_x=8,
            tiles_y=8,  # Standard 8x8 grid
            clip_limit=2.0,  # Moderate contrast limiting
        )

        # 2. Aggressive enhancement - more contrast, more local adaptation
        clahe_aggressive = fn.clahe(
            base_image,
            tiles_x=16,
            tiles_y=16,  # Finer 16x16 grid
            clip_limit=4.0,  # Higher contrast limit
        )

        # 3. Gentle enhancement - subtle improvement
        clahe_gentle = fn.clahe(
            base_image,
            tiles_x=4,
            tiles_y=4,  # Coarser 4x4 grid
            clip_limit=1.0,  # Conservative contrast limit
        )

        return base_image, clahe_default, clahe_aggressive, clahe_gentle

    return parameter_demo_pipeline()

## 🚀 Running the CLAHE Pipeline

Now let's execute our pipeline and see CLAHE in action! We'll analyze the results and measure the contrast improvement.

In [ ]:
# 🏗️ Create and build pipeline
print("🔧 Creating CLAHE pipeline...")
pipe = create_clahe_pipeline(batch_size=2, num_threads=1, device_id=0)
pipe.build()
print("✅ Pipeline built successfully")

# 🎬 Run pipeline
print("\n🎬 Running pipeline...")
outputs = pipe.run()
original_images, clahe_images = outputs

# 📥 Move to CPU for analysis
original_batch = original_images.as_cpu()
clahe_batch = clahe_images.as_cpu()

print(f"✅ Processed {len(original_batch)} images")

# 📊 Analyze results
print("\n" + "=" * 50)
print("📊 CLAHE RESULTS ANALYSIS")
print("=" * 50)

for i in range(len(original_batch)):
    original = np.array(original_batch[i])
    enhanced = np.array(clahe_batch[i])

    print(f"\n🖼️  Image {i + 1}:")
    print(
        f"  📏 Original  - Shape: {original.shape}, Range: [{original.min():.1f}, {original.max():.1f}]"
    )
    print(
        f"  ✨ Enhanced  - Shape: {enhanced.shape}, Range: [{enhanced.min():.1f}, {enhanced.max():.1f}]"
    )

    # Calculate contrast metrics (standard deviation as a proxy for contrast)
    orig_std = np.std(original)
    enhanced_std = np.std(enhanced)
    contrast_improvement = enhanced_std / orig_std if orig_std > 0 else 1.0

    print(f"  📈 Contrast improvement: {contrast_improvement:.2f}x")

print("\n🎉 CLAHE pipeline executed successfully!")

## 🎛️ Parameter Comparison Experiment

Let's compare different CLAHE parameter settings to understand their effects on image enhancement.

In [ ]:
# 🔬 Demonstrate parameter variations
print("🔬 Testing different CLAHE parameters...")
param_pipe = demonstrate_clahe_parameters()
param_pipe.build()

param_outputs = param_pipe.run()
base, default, aggressive, gentle = param_outputs

# Convert to numpy arrays for analysis
base_img = np.array(base.as_cpu()[0])
default_img = np.array(default.as_cpu()[0])
aggressive_img = np.array(aggressive.as_cpu()[0])
gentle_img = np.array(gentle.as_cpu()[0])

# 📊 Compare the results
print("\n" + "=" * 60)
print("🎛️  PARAMETER COMPARISON RESULTS")
print("=" * 60)

configurations = [
    ("🔸 Base image (no CLAHE)", base_img),
    ("⚖️  Default CLAHE (8x8, limit=2.0)", default_img),
    ("🔥 Aggressive CLAHE (16x16, limit=4.0)", aggressive_img),
    ("🌸 Gentle CLAHE (4x4, limit=1.0)", gentle_img),
]

for name, img in configurations:
    std_dev = np.std(img)
    print(f"{name}")
    print(f"   📊 Standard deviation (contrast measure): {std_dev:.2f}")
    print()

print("💡 Key Takeaways:")
print("   • Higher std dev = more contrast")
print("   • More tiles (16x16) = more local adaptation")
print("   • Higher clip limit = stronger enhancement")
print("   • Choose parameters based on your image type and requirements!")

## 🎯 Practical Applications & Next Steps

### Where to Use CLAHE:
- 🏥 **Medical Imaging**: Enhance X-rays, CT scans, MRI images
- 👁️ **Computer Vision**: Improve object detection in low-contrast scenes  
- 📸 **Photography**: Enhance details in shadows and highlights
- 🛡️ **Security**: Improve visibility in surveillance footage
- 🌌 **Astronomy**: Enhance celestial object visibility

### Parameter Tuning Guidelines:

| Image Type | Recommended tiles_x/y | Recommended clip_limit | Notes |
|------------|----------------------|----------------------|-------|
| Medical scans | 8-12 | 1.5-2.5 | Preserve diagnostic details |
| Natural photos | 6-10 | 2.0-3.0 | Balance enhancement and naturalness |
| Low-light images | 10-16 | 3.0-4.0 | Aggressive enhancement acceptable |
| High-noise images | 4-8 | 1.0-2.0 | Avoid amplifying noise |

### Performance Tips:
- 🚀 Use `device="gpu"` for maximum performance
- 📦 Process images in batches when possible
- 🎨 Set `luma_only=True` for RGB images to preserve color balance
- 🔧 Experiment with parameters on representative samples

### Try These Experiments:
1. **Real Images**: Replace synthetic data with your own image directory
2. **Video Processing**: Apply CLAHE to video frames in a sequence
3. **Multi-scale Enhancement**: Combine CLAHE with other DALI operators
4. **Benchmarking**: Compare CLAHE performance vs. other enhancement methods